# Identify Lines with Bad Fits

In [ ]:
from glob import glob
from scipy.optimize import least_squares
import pickle
import seaborn as sns

from waveCal import *

## Gather Files

In [ ]:
# LFC
lfc_files = glob('/mnt/home/lzhao/ceph/lfc5*/LFC_*.fits')
ckpt_files = glob('/mnt/home/lzhao/ceph/ckpt5*/LFC_19*.npy')
lfc_files, lfc_times = sortFiles(lfc_files, get_mjd=True)
ckpt_files = sortFiles(ckpt_files)
num_lfc_files = len(lfc_files)

hdus = fits.open(lfc_files[0])
t_spec = hdus[1].data['spectrum'].copy()
t_errs = hdus[1].data['uncertainty'].copy()
t_mask = hdus[1].data['pixel_mask'].copy()
hdus.close()
nord, npix = t_spec.shape

lfc_orders = range(45,76)

In [ ]:
# ThAr
thar_files = glob('/mnt/home/lzhao/ceph/thar5*/ThAr_*.fits')
thid_files  = glob('/mnt/home/lzhao/ceph/thid5*/ThAr_*.thid')
thar_files, thar_times = sortFiles(thar_files, get_mjd=True)
thar_files = thar_files[1:] # First file is from before LFCs
thar_times = thar_times[1:]
thid_files = sortFiles(thid_files) [1:]
num_thar_files = len(thar_files)

## Identify Lines and Predictions

In [ ]:
lfc_lines = pickle.load(open('./191220_ckptPatch9.pkl','rb'))
thar_lines = pickle.load(open('./191220_thidPatch15.pkl','rb'))

## Find Chi of Line Fits

In [ ]:
lfc_resid = (lfc_lines['x_values']-lfc_lines['denoised_xs'])
thar_resid = (thar_lines['x_values']-thar_lines['denoised_xs'])

In [ ]:
plt.figure(figsize=(6.4*2,4.8))
plt.title('Residual of LFC Lines')
plt.xlabel('Line Number')
plt.ylabel('Residual [pixels]')

# Draw order boundaries
old = 0
for i,o in enumerate(lfc_lines['orders']):
    if o!=old:
        plt.axvline(i,color='0.75')
        old=o
plt.axvline(i+1,color='0.75')

for i in tqdm(range(len(lfc_resid))):
    plt.plot(lfc_resid[i],'k.',alpha=0.1)
plt.savefig('./Figures/191220_lfcResids.png')

In [ ]:
plt.figure(figsize=(6.4*2,4.8))
plt.title('Residual of ThAr Lines')
plt.xlabel('Line Number')
plt.ylabel('Residual [pixels]')

# Draw order boundaries
old = 0
for i,o in enumerate(thar_lines['orders']):
    if o!=old:
        plt.axvline(i,color='0.75')
        old=o
plt.axvline(i+1,color='0.75')

for i in tqdm(range(len(thar_resid))):
    plt.plot(thar_resid[i],'k.',alpha=0.1)
plt.savefig('./Figures/191220_tharResids.png')

In [ ]:
lfc_chi = lfc_resid/np.sqrt(lfc_lines['x_errors'])
thar_chi = thar_resid/np.std(thar_resid,axis=0)

In [ ]:
plt.figure()
plt.title('Distribution of LFC Errors')
plt.xlabel('Error')
plt.ylabel('Normalized Frequency')

a,b,c = plt.hist(lfc_lines['x_errors'].flatten(),50,
                 density=True,histtype='step',label='Reported Errors');
plt.hist(np.nanstd(lfc_resid,axis=0),bins=b,
         density=True,histtype='step',label='std of Line');
plt.legend()
plt.tight_layout()
plt.savefig('./Figures/191220_lfcErrs.png')

In [ ]:
plt.figure(figsize=(6.4*2,4.8))
plt.title('Chi of LFC Lines')
plt.xlabel('Line Number')
plt.ylabel('Chi')

# Draw order boundaries
old = 0
for i,o in enumerate(lfc_lines['orders']):
    if o!=old:
        plt.axvline(i,color='0.75')
        old=o
plt.axvline(i+1,color='0.75')

for i in tqdm(range(len(lfc_chi))):
    plt.plot(lfc_chi[i],'k.',alpha=0.05)
plt.axhline(3,color='r')
plt.axhline(-3,color='r')
plt.savefig('./Figures/191220_lfcChis.png')

In [ ]:
plt.figure(figsize=(6.4*2,4.8))
plt.title('Chi of LFC Lines')
plt.xlabel('Line Number')
plt.ylabel('Chi')

# Draw order boundaries
old = 0
for i,o in enumerate(lfc_lines['orders']):
    if o!=old:
        plt.axvline(i,color='0.75')
        old=o
plt.axvline(i+1,color='0.75')

for i in tqdm(range(len(lfc_chi))):
    plt.plot(lfc_resid[i]/np.std(lfc_resid,axis=0),'k.',alpha=0.05)
plt.axhline(3,color='r')
plt.axhline(-3,color='r')
plt.savefig('./Figures/191220_lfcChis_cheat.png')

In [ ]:
plt.figure(figsize=(6.4*2,4.8))
plt.title('Chi of ThAr Lines')
plt.xlabel('Line Number')
plt.ylabel('Chi')

# Draw order boundaries
old = 0
for i,o in enumerate(thar_lines['orders']):
    if o!=old:
        plt.axvline(i,color='0.75')
        old=o
plt.axvline(i+1,color='0.75')

for i in tqdm(range(len(thar_chi))):
    plt.plot(thar_chi[i],'k.',alpha=0.1)
plt.axhline(3,color='r')
plt.axhline(-3,color='r')
plt.savefig('./Figures/191220_tharChis.png')